# API Code Snippet for Parsing Address to Lat,Lon
* Note1: This code snippet serves as a reusable pattern for future AI API integrations, should cost considerations be removed.
* Note2: This is based on Gemini API
* The huge blocker was this error message below:

"Error Message: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '250'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '30s'}]}}"

## Authenticate

In [ ]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
from google import genai
from google.genai import errors
import numpy as np
import pandas as pd

# Initialize your Gemini AI key
client = genai.Client(api_key='YOUR_KEY_HERE')

## Example of generating content using Gemini AI's API
# response = client.models.generate_content(
#     model="gemini-2.5-flash", contents="Say hello to the world!"
# )
# print(response.text)

: 

In [ ]:
def generate_lat_long(latitude, longitude, borough, zip_code,
                      on_street_name, off_street_name, cross_street_name):
    """
    This function calls the Gemini API to infer (latitude, longitude)
    with retries for API failures.
    """
    input_data_str = f"""
Input:
latitude: {latitude if pd.notna(latitude) else 'None'}
longitude: {longitude if pd.notna(longitude) else 'None'}
borough: {borough if pd.notna(borough) else 'None'}
zip_code: {zip_code if pd.notna(zip_code) else 'None'}
on_street_name: {on_street_name if pd.notna(on_street_name) else 'None'}
off_street_name: {off_street_name if pd.notna(off_street_name) else 'None'}
cross_street_name: {cross_street_name if pd.notna(cross_street_name) else 'None'}
"""
    base_prompt = """
You are a geospatial coordinate resolver for motor vehicle collisions in NYC.
Given location arguments, determine the most accurate (latitude, longitude) pair as a plain text tuple.
Arguments (may be blank/None/NaN): latitude, longitude, borough, zip_code, on_street_name, off_street_name, cross_street_name.

Resolution Logic:
1. Prioritize direct coordinates if valid.
2. Infer from address (borough, zip_code, streets) if coordinates are missing/invalid. Use internal knowledge for geocoding, prioritizing specific components.
3. Return (None, None) if confident coordinates cannot be determined.

---
**YOUR SPECIFIC TASK IS TO PROVIDE THE COORDINATE FOR THE FINAL "INPUT:" BLOCK ONLY.**
**Your output MUST be ONLY one (latitude, longitude) tuple string. For example: (40.1234, -74.5678)**

---
Examples of input data structure (DO NOT provide output for these examples):
Input:
latitude: 40.7128
longitude: -74.0060
borough: Manhattan
zip_code: 10007
on_street_name: Broadway
off_street_name: Vesey St
cross_street_name: None

Input:
latitude: None
longitude: None
borough: Brooklyn
zip_code: 11201
on_street_name: Main St
off_street_name: Front St
cross_street_name: None

Input:
latitude: None
longitude: -73.95
borough: Queens
zip_code: None
on_street_name: Astoria Blvd
off_street_name: None
cross_street_name: None

Input:
latitude: None
longitude: None
borough: None
zip_code: None
on_street_name: Unknown Street
off_street_name: Nowhere Ave
cross_street_name: None
"""
    full_prompt = base_prompt + input_data_str
    
    max_retries, wait_time = 3, 60
    raw_response_string = ""
    
    for attempt in range(max_retries):
        try:
            response = (client.models.generate_content if hasattr(client, 'models') else client.generate_content)(
                model="gemini-2.5-flash", contents=full_prompt
            )
            raw_response_string = response.text.strip()
            break # Success, exit loop
        # --- FIX APPLIED HERE: Changed genai.types.APIError to genai.APIError ---
        except errors.APIError as e:
            if hasattr(e, 'response') and hasattr(e.response, 'status_code') and e.response.status_code == 429:
                print(f"  API: Rate limit ({attempt+1}/{max_retries}). Waiting {wait_time}s. Error Message: {e}")
                time.sleep(wait_time)
            else:
                # Catch other API errors that are not rate limits
                print(f"  API Error (non-429): {e}. Returning NaN."); return np.nan, np.nan
        except Exception as e:
            print(f"  Unexpected error ({attempt+1}/{max_retries}): {e}. Returning NaN."); return np.nan, np.nan
    else: # All retries failed
        print(f"  Max retries ({max_retries}) exhausted. Returning NaN."); return np.nan, np.nan

    # --- Simplified Parsing: Directly evaluate the response string ---
    lat_res, lon_res = np.nan, np.nan
    
    try:
        p_tuple = ast.literal_eval(raw_response_string)
        
        if isinstance(p_tuple, tuple) and len(p_tuple) == 2:
            lat_res = float(p_tuple[0]) if p_tuple[0] is not None else np.nan
            lon_res = float(p_tuple[1]) if p_tuple[1] is not None else np.nan
        else:
            print(f"  Warn: Malformed or non-2-element tuple output: '{p_tuple}'. Raw: '{raw_response_string}'")
    except (ValueError, SyntaxError) as e:
        print(f"  Error parsing response as tuple: {e}. Raw: '{raw_response_string}'")
    except Exception as e: # Catch any other unexpected parsing errors
        print(f"  Unexpected parsing error: {e}. Raw: '{raw_response_string}'")

    return lat_res, lon_res

## Code to Populate DataFrame

In [ ]:
# Create column for AI to fill in lat/long
collision_df['Lat_x_AI'] = np.nan
collision_df['Lon_y_AI'] = np.nan
        
for index,row in collision_df.loc[collision_df['flag_orig_latlong_missing'] == True].iterrows():
        print(f"Working on index {index} (Collision ID: {row['collision_id']})")
        response = generate_lat_long(
            row['latitude'],
            row['longitude'],
            row['borough'],
            row['zip_code'],
            row['on_street_name'],
            row['off_street_name'],
            row['cross_street_name']
        )
        
        # Apply AI Generated lat/long to the DataFrame
        lat_ai, lon_ai = response[0], response[1]

        collision_df.loc[index, 'Lat_x_AI'] = lat_ai
        collision_df.loc[index, 'Lon_y_AI'] = lon_ai

        # You can print progress
        print(f"Processed index {index}. AI Coords: ({lat_ai}, {lon_ai})")

print("\nFinished processing. Check 'Lat_x_AI' and 'Lon_y_AI' columns.")
collision_df.head()